Origin of the datafiles:

https://iers-conventions.obspm.fr/content/chapter5/additional_info/

In [6]:
# Setup the regex to identify sections in the files.
import re
section_pattern = re.compile(r'^\s*j = (\d*)\s*Number of terms = (\d*)\s*$')

# Template for the header file output.
HPP_FILE_TMPLT = r"""#ifndef HEYOKA_DETAIL_IAU2000_{0}_HPP
#define HEYOKA_DETAIL_IAU2000_{0}_HPP

#include <cstddef>
#include <cstdint>

#include <heyoka/config.hpp>
#include <heyoka/mdspan.hpp>

HEYOKA_BEGIN_NAMESPACE

namespace model::detail
{{

{1}

}} // namespace model::detail

HEYOKA_END_NAMESPACE

#endif

"""

# Template for the cpp file output.
CPP_FILE_TMPLT = r"""#include <cstddef>
#include <cstdint>

#include <heyoka/config.hpp>
#include <heyoka/detail/iau2000/{0}.hpp>
#include <heyoka/mdspan.hpp>

HEYOKA_BEGIN_NAMESPACE

namespace model::detail
{{

namespace
{{

{1}

}}

{2}

}} // namespace model::detail

HEYOKA_END_NAMESPACE

"""

In [16]:
def parse_iau2000_file(input_name: str, output_basename: str) -> None:
    # Open the file, read line-by-line.
    with open(input_name, 'r') as f:
        lines = f.readlines()

    # Look for the sections.
    sec_descs = []
    for idx, line in enumerate(lines):
        m = section_pattern.match(line)
        if m:
            sec_descs.append((idx, int(m.groups()[0]), int(m.groups()[1])))

    hpp_content = ""
    cpp_content1 = ""
    cpp_content2 = ""
    
    for sec_idx, desc in enumerate(sec_descs):
        start, j_val, n_terms = desc
        # NOTE: the data starts 2 lines after the section header.
        begin = start+2

        # Setup the hpp content for this section.
        hpp_content += f"extern const mdspan<const std::int8_t, extents<std::size_t, {n_terms}, 14>> iau2000_{output_basename}_args_idxs_{j_val};\n"
        hpp_content += f"extern const mdspan<const double, extents<std::size_t, {n_terms}, 2>> iau2000_{output_basename}_cfs_{j_val};\n\n"

        # Compute the names of the data variables for the current section.
        args_data_name = f"iau2000_{output_basename}_args_idxs_data_{j_val}"
        cfs_data_name = f"iau2000_{output_basename}_args_cfs_data_{j_val}"

        # Create the code to initialise the spans.
        cpp_content2 += f"const mdspan<const std::int8_t, extents<std::size_t, {n_terms}, 14>> iau2000_{output_basename}_args_idxs_{j_val}({args_data_name});\n"
        cpp_content2 += f"const mdspan<const double, extents<std::size_t, {n_terms}, 2>> iau2000_{output_basename}_cfs_{j_val}({cfs_data_name});\n\n"

        # Build the data variables.
        cur_args = f"const std::int8_t {args_data_name}[{n_terms*14}ul] = {{";
        cur_cfs = f"const double {cfs_data_name}[{n_terms*2}ul] = {{";
        for line_idx in range(begin, begin + n_terms):
            line_split = lines[line_idx].split()
            assert len(line_split) == 17

            cur_args += ','.join(line_split[3:17]) + ','
            cur_cfs += ','.join(line_split[1:3]) + ','
        cur_args += '};\n\n';
        cur_cfs += '};\n\n';

        # Update cpp_content1.
        cpp_content1 += cur_args + cur_cfs

    # Write to the output files.
    with open(f'{output_basename}.hpp', 'w') as f:
        f.write(HPP_FILE_TMPLT.format(output_basename.upper(), hpp_content))

    with open(f'{output_basename}.cpp', 'w') as f:
        f.write(CPP_FILE_TMPLT.format(output_basename, cpp_content1, cpp_content2))

## tab5.2a.txt

In [17]:
parse_iau2000_file('tab5.2a.txt', 'X')

In [18]:
parse_iau2000_file('tab5.2b.txt', 'Y')

In [19]:
parse_iau2000_file('tab5.2d.txt', 's')